# 3.5. Concise Implementation of Linear Regression

In [ ]:
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

## 3.5.1. Defining the Model

In [ ]:
# In PyTorch, the fully connected layer is defined in Linear and LazyLinear classes
class LinearRegression(d2l.Module):
    def __init__(self, lr):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.LazyLinear(1)
        self.net.weight.data.normal_(0, 0.01)
        self.net.bias.data.fill_(0)

In [ ]:
@d2l.add_to_class(LinearRegression)
def forward(self, x):
    return self.net(x)

## 3.5.2. Defining the Loss Function

In [ ]:
@d2l.add_to_class(LinearRegression)
def loss(self, y_hat, y):
    fn = nn.MSELoss()
    # fn = nn.HuberLoss()
    return fn(y_hat, y)

# 卧槽？一开始写成直接return nn.MSELoss(y_hat, y)，会报错，难道不是一个意思？？？？

## 3.5.3. Defining the Optimization Algorithm

In [ ]:
@d2l.add_to_class(LinearRegression)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), self.lr)

## 3.5.4. Training

In [ ]:
model = LinearRegression(lr = 0.03)
data = d2l.SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
trainer = d2l.Trainer(max_epochs=3)
trainer.fit(model, data)

In [ ]:
@d2l.add_to_class(LinearRegression)  #@save
def get_w_b(self):
    return (self.net.weight.data, self.net.bias.data)
w, b = model.get_w_b()

print(f'error in estimating w: {data.w - w.reshape(data.w.shape)}')
print(f'error in estimating b: {data.b - b}')

## 3.5.6. Exercises

1. How would you need to change the learning rate if you replace the aggregate loss over the minibatch with an average over the loss on the minibatch?

是的，当你从**总和的 loss（aggregate loss）** 切换到 **平均的 loss（average loss）** 时，loss 值的确会变小，因为它被 **批量大小（batch size）** 缩放了。举个例子：

- **总和的 loss**：对每个 minibatch 的样本计算 loss，然后对所有样本的 loss 取总和。
- **平均的 loss**：对每个 minibatch 的样本计算 loss，然后对这些 loss 取平均。

由于**平均 loss = 总和 loss / 批量大小**，所以平均 loss 会比总和 loss 小，具体来说，小了一个批量大小的倍数。

但是在实际训练过程中，这个缩放并不影响整体的训练效果，因为：

- **梯度**：loss 越大，梯度也会越大。总和 loss 下的梯度值较大，而在平均 loss 下的梯度值较小。平均 loss 缩小了梯度的规模，但与 minibatch 大小无关。换句话说，平均 loss 可以让梯度保持一致的尺度，不依赖于批量大小。
  
- **学习率调整**：正因为平均 loss 下梯度的缩放是由批量大小导致的，这种变化是固定的，所以通常不需要调整学习率。如果之前使用总和 loss，现在使用平均 loss，梯度会自动缩小一个批量大小的比例，但是在大多数情况下，原本的学习率就能应付这种变化。

### 总结
虽然 loss 变小了，但相应的梯度也按比例缩小，因此一般**不需要改变学习率**。如果你原来使用的是总和 loss，现在切换到平均 loss，权重更新的步伐保持稳定，不需要调整学习率。

2. Review the framework documentation to see which loss functions are provided. In particular, replace the squared loss with Huber’s robust loss function. That is, use the loss function

Huber 损失函数的可复制版本公式如下：

$$
L_{\delta}(r) =
\begin{cases} 
\frac{1}{2} r^2 & \text{for } |r| \leq \delta, \\
\delta (|r| - \frac{1}{2} \delta) & \text{for } |r| > \delta,
\end{cases}
$$

其中：
- $ r = y - \hat{y} $ 是残差；
- $ \delta $ 是控制阈值的参数。

